## A Simulated IOT Federated Learning Framework for Forest Fire Prediction

Forest fire ignition prediction is essential to the safety of communities and the proper allocation of fire fighting resources. This notebook simulates a network of clients that collect local meteorological data. Each client participates in a federated learning system where a client trains a machine learning model to classify whether the area will have a forest fire ignition. The model weights of each client are shared with a central server that averages the weights and shares them back to the clients.

This notebook demonstrates the three machine learning types shown below.

|<center>name|<center>server|<center>clients|<center>training|<center>training data|<center>evaluation type|<center>evaluation|<center>evaluation data|
|---|---|---|---|---|---|---|---|
|<center>classic machine learning|<center>yes|<center>no|<center>on server|<center>on server|<center>centralized|<center>on server|<center>on server|
|<center>federated machine learning (Central Eval)|<center>yes|<center>yes|<center>on clients|<center>on clients|<center>centralized|<center>on server|<center>on server|
|<center>federated machine learning (Federated Eval)|<center>yes|<center>yes|<center>on clients|<center>on clients|<center>distributed|<center>on clients|<center>on clients|


The data used is from the paper: *Framework for Creating Forest Fire Ignition Prediction Datasets.* Each row represents meteorological data at a geographical location at a specific time. TODO: Add table example.

Much of the code used in this notebook is based on the Flower code examples located [here](https://github.com/adap/flower/tree/main/examples) and the Keras timeseries tutorials located [here](https://keras.io/examples/timeseries/).
The code below is very much a work in progress. 

In [1]:
#if this file is being used in colab set to 1 otherwise 0
using_colab = 1

In [2]:
if (using_colab == 1):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
if (using_colab ==1):
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
#load libraries
import math
import os
import glob
import gc
import datetime
import typing
from typing import List
from typing import Tuple
from typing import Dict
from typing import Optional
import random
import tempfile
from tables.file import File

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Model

In [5]:
if (using_colab == 1):
    !pip install -q flwr[simulation] pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14

In [6]:
import flwr as fl
from flwr.common import Metrics

In [7]:
#overall environment settings

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Training on GPU or CPU?
#tf.config.set_visible_devices([], 'CPU')
print(
    f"Training on {'GPU' if tf.config.get_visible_devices('GPU') else 'CPU'} using TensorFlow {tf.__version__} and Flower {fl.__version__}"
)

Training on CPU using TensorFlow 2.12.0 and Flower 1.4.0


In [38]:
#global variables

ml_type = 0 # classic ML = 0, federated ML w/ centralized evaluation = 1, federated ML w/ federated eval = 2

federated_path = "<path to your client datasets>" 
centralized_path = "<path to your server dataset"
results_path = "<path to where you want to store results>"


cid = str(0) # preliminary client id
log_dir = "<path to where you want to store Tensorflow logs>" + cid + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
# log_dir = "./logs/fit/" + cid + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

downsample_test_set = 0 # 0 if test set not downsampled, 1 otherwise

# TODO: Add the below to a client config file
num_rounds = 20
epochs = 10 # the number of epochs classic machine learning will use
fl_epochs = 3
sequence_len = 120 # 5 days * 24 hours
past_len = sequence_len
future_len = 24 # 1*24 hours
sampling_rate = 1 # in time series conversion use every row (hour) in loaded datasets
sequence_stride = 1 # in time series conversion each series is this far apart
fraction_fit = 0.6 # fraction of clients to use per training round
fraction_evaluate = 0.3 # fraction of clients to use per evaluation round

#create an array for results
np.set_printoptions(suppress=True)
results = np.empty([13,3])

## Data loading functions

In [9]:
def normalize_data(x):
    """Normalizes the data of an array by column. 
    Shifts and scales inputs into a distribution centered around 0 
    with standard deviation 1.

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        The original array, but normalized.
    """
    data = x
    layer = layers.Normalization()
    layer.adapt(data)
    features_normalized = layer(data)
    return features_normalized

In [10]:
def mask_create(x):
    """Finds the class count of the input array and creates a mask that can be used
    to randomly downsample an array of labels so that the number of 
    negative labels = the number of positive labels. 

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        A masked version of the input array.
    """
    mask_length = x.shape[0]
    mask = tf.reshape(x, [mask_length])
    y, idx, class_count = tf.unique_with_counts(mask)
    ignition_count = tf.get_static_value(class_count[1])
    mask = mask.numpy()
    count = 0 
    while count < ignition_count:
        #rand_num = random.randint(0,mask_length)
        rand_num = random.randint(1, mask_length-1)
        if (mask[rand_num] == 0):
            mask[rand_num] = 1
            count += 1
    return mask

In [11]:
def load_datasets(path: str):
    """Loads all the csv datasets in a folder.
    The loaded data is divided into train, validation, and test sets.
    The data is turned into time series data
    All the data is normalized.
    Train and validation datasets are downsampled.
    TODO: divide this function into smaller functions

    Parameters
    ----------
    path: string 
        The path to the dataset folder.
        
    Returns
    -------
    train_x, train_y, val_x, val_y, test_x, test_y : NDarrays
        A masked version of the input array.
    """

    train_x = []
    train_y = []
    val_x = []
    val_y = []
    test_x = []
    test_y = []

    #load data
    for filename in glob.glob(os.path.join(path, '*.csv')):
        print("\nnow reading " + filename + "\n")
        #read file
        df = pd.read_csv(filename, index_col=[0])
        
        df_train = df[(df['year'] < 2001)]
        df_val = df[(df['year'] > 2001) & (df['year'] < 2012)]
        df_test = df[(df['year'] >= 2012)]
        
        features = ['stl2', 't2m', 'stl1', 'stl3', 'skt', 'swvl1', 'd2m', 'swvl2']
        train_features = df_train[features]
        train_labels = df_train[["ignition"]]
        val_features = df_val[features]
        val_labels = df_val[["ignition"]]
        test_features = df_test[features]
        test_labels = df_test[["ignition"]]
        #convert to numpy
        train_features = train_features.values
        val_features = val_features.values
        test_features = test_features.values

        #normalize
        train_features_normalize = normalize_data(train_features)
        val_features_normalize = normalize_data(val_features)
        test_features_normalize = normalize_data(test_features)
        
        #we want to predict at a future point
        #so we clip the length of the features plus the hours till the future point
        start = past_len + future_len
        train_labels = train_labels.iloc[start:].values
        val_labels = val_labels.iloc[start:].values
        test_labels = test_labels.iloc[start:].values
        
        batch_size = 107856 #factor of 5136 (321 * 16)
        #convert to time series data
        train_dataset = keras.utils.timeseries_dataset_from_array(
            train_features_normalize,
            train_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        val_dataset = keras.utils.timeseries_dataset_from_array(
            val_features_normalize,
            val_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        test_dataset = keras.utils.timeseries_dataset_from_array(
            test_features_normalize,
            test_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)
        
        #for bookkeeping print out the shapes of the datasets
        for train_features, train_labels in train_dataset:
            print("train_dataset features shape:", train_features.shape)
            print("targets_dataset labels shape:", train_labels.shape)
            break

        for val_features, val_labels in val_dataset:
            print("\nval_dataset features shape:", val_features.shape)
            print("val_dataset labels shape:", val_labels.shape)
            break

        for test_features, test_labels in test_dataset:
            print("\ntest_dataset features shape:", test_features.shape)
            print("test_dataset labels shape:", test_labels.shape)
            break
       
        # randomly downsample the data using masks
        train_mask = mask_create(train_labels)
        train_features_masked = tf.boolean_mask(train_features, train_mask)
        train_labels_masked = tf.boolean_mask(train_labels, train_mask)
        
        val_mask = mask_create(val_labels)
        val_features_masked = tf.boolean_mask(val_features, val_mask)
        val_labels_masked = tf.boolean_mask(val_labels, val_mask)
        
        test_mask = mask_create(test_labels)
        test_features_masked = tf.boolean_mask(test_features, test_mask)
        test_labels_masked = tf.boolean_mask(test_labels, test_mask)
        
        train_x.append(train_features_masked)
        train_y.append(train_labels_masked)
        val_x.append(val_features_masked)
        val_y.append(val_labels_masked)
        if (downsample_test_set == 1):
            test_x.append(test_features_masked)
            test_y.append(test_labels_masked)
        else:
            test_x.append(test_features)
            test_y.append(test_labels)
        
    print("\nDone loading data.\n")
    return train_x, train_y, val_x, val_y, test_x, test_y 



In [12]:
def get_value_count(x):
    """A helper function that returns the count of class labels.

    Parameters
    ----------
    x: NDArray 
        An array with class labels.
        
    Returns
    -------
    non_ignition_count, ignition_count : int
        The counts of the ignition class.
    """
    length = x[0].shape[0]
    x = tf.reshape(x, [length])
    y, idx, class_count = tf.unique_with_counts(x)
    non_ignition_count = tf.get_static_value(class_count[0])
    ignition_count = tf.get_static_value(class_count[1])
    return non_ignition_count, ignition_count
    

## Model and Metrics

In [13]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
]

def make_model(metrics=METRICS):
    inputs = keras.Input(shape=(sequence_len, trainloaders_x[0].shape[2]))
    x = layers.LSTM(8, activation='sigmoid')(inputs)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)
        
    return model


## Federated Learning Functions

In [14]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val, x_test, y_test, tb_callback) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
        self.x_test, self.y_test = x_test, y_test
        self.tb_callback = tb_callback

    def get_parameters(self, config):
        return self.model.get_weights()

    # presetting config allows us to use FlowerClient with classic ml.
    def fit(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        print("in fit")
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=epochs, verbose=2, callbacks=self.tb_callback, validation_data=(self.x_val, self.y_val),)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        self.model.set_weights(parameters)
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc  = self.model.evaluate(self.x_test, self.y_test, verbose=2)
        return loss, len(self.x_val), {"tp": tp,
                                       "fp": fp,
                                       "tn": tn,
                                       "fn": fn,
                                       "accuracy": accuracy,
                                       "precision": precision,
                                       "recall": recall,
                                       "auc": auc,
                                       "prc": prc
                                      }

    # this method allows metrics to be passed when using a single server with no clients
    def evaluate_single(self, x_test, y_test):
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = self.model.evaluate(x_test, y_test, verbose=2)
        return loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc

In [15]:
def client_fn(cid: str) -> fl.client.Client:

    print("\nThis is client: ", cid)

    x_train_cid = trainloaders_x[int(cid)]
    y_train_cid = trainloaders_y[int(cid)]
    x_val_cid = valloaders_x[int(cid)]
    y_val_cid = valloaders_y[int(cid)]
    x_test_cid = testloaders_x[int(cid)]
    y_test_cid = testloaders_y[int(cid)]

    print("Loaded data for client: ", cid, "\n")

    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
    ]

    def make_model(metrics=METRICS):
        inputs = keras.Input(shape=(sequence_len, x_train_cid.shape[2]))
        x = layers.LSTM(8, activation='sigmoid')(inputs)
        x = layers.Flatten()(x)
        outputs = layers.Dense(1, activation="sigmoid")(x)
        model = keras.Model(inputs, outputs)

        model.compile(
            optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics)
        
        return model
    
    print("Making model: ", cid)
    model = make_model()
    
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    tb_callback = keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)

    # Create and return client
    print("\nClient CID: " + str(cid) + " is done.\n")
    return FlowerClient(model, x_train_cid, y_train_cid, x_val_cid, y_val_cid, x_test_cid, y_test_cid, tensorboard)

## Launch classic machine learning

In [16]:
#TODO: there is a random out-of-bounds error with masking, for now if it occurs run this cell again
#load the dataset for centralized evaluation (for classic ml training and testing) 
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(centralized_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/01_clients/dly_avg_1of1_50.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

Done loading data.



In [17]:
#save copy for federated centralized evaluation
central_testloaders_x = testloaders_x.copy()
central_testloaders_y = testloaders_y.copy()

In [18]:
count = get_value_count(trainloaders_y)
print("Train set nonignitions and ignitions are:", count)
count = get_value_count(valloaders_y)
print("Validation set nonignitions and ignitions are:", count)
count = get_value_count(testloaders_y)
print("Test set nonignitions and ignitions are:", count)
count = get_value_count(central_testloaders_y)
print("Central test set nonignitions and ignitions are:", count)

Train set nonignitions and ignitions are: (2811, 2811)
Validation set nonignitions and ignitions are: (2534, 2534)
Test set nonignitions and ignitions are: (44719, 1361)
Central test set nonignitions and ignitions are: (44719, 1361)


In [19]:
#create an initial model to prime flower client
model = make_model()

In [20]:
#train single server classic ml
#create a flower client that represents a classic ml single server
central_server_model = client_fn(str(0))
#set parameters for classic_ml so it can use single flower client function
parameters = model.get_weights()
#run fit
history = central_server_model.fit(parameters)


This is client:  0
Loaded data for client:  0 

Making model:  0

Client CID: 0 is done.

in fit
Epoch 1/10
176/176 - 10s - loss: 0.6484 - tp: 1660.0000 - fp: 855.0000 - tn: 1956.0000 - fn: 1151.0000 - accuracy: 0.6432 - precision: 0.6600 - recall: 0.5905 - auc: 0.6960 - prc: 0.6850 - val_loss: 0.6278 - val_tp: 1778.0000 - val_fp: 883.0000 - val_tn: 1651.0000 - val_fn: 756.0000 - val_accuracy: 0.6766 - val_precision: 0.6682 - val_recall: 0.7017 - val_auc: 0.7117 - val_prc: 0.6844 - 10s/epoch - 60ms/step
Epoch 2/10
176/176 - 8s - loss: 0.6224 - tp: 1912.0000 - fp: 979.0000 - tn: 1832.0000 - fn: 899.0000 - accuracy: 0.6660 - precision: 0.6614 - recall: 0.6802 - auc: 0.7199 - prc: 0.7139 - val_loss: 0.6171 - val_tp: 1782.0000 - val_fp: 858.0000 - val_tn: 1676.0000 - val_fn: 752.0000 - val_accuracy: 0.6823 - val_precision: 0.6750 - val_recall: 0.7032 - val_auc: 0.7218 - val_prc: 0.6982 - 8s/epoch - 43ms/step
Epoch 3/10
176/176 - 8s - loss: 0.6161 - tp: 1871.0000 - fp: 900.0000 - tn: 1911.

In [21]:
#evaluate single server classic ml
loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = central_server_model.evaluate_single(central_testloaders_x[0], central_testloaders_y[0])

1440/1440 - 12s - loss: 0.6018 - tp: 943.0000 - fp: 13020.0000 - tn: 31699.0000 - fn: 418.0000 - accuracy: 0.7084 - precision: 0.0675 - recall: 0.6929 - auc: 0.7628 - prc: 0.0835 - 12s/epoch - 8ms/step


In [22]:
print("Single server classic ml evaluation\n\
\ttp:\t%d\n\
\tfp:\t%d\n\
\ttn:\t%d\n\
\tfn:\t%d\n\n\
\tloss:\t%f\n\
\tacc:\t%f\n\
\tprec:\t%f\n\
\trec:\t%f\n\
\tauc:\t%f\n\
\tprc:\t%f\
" % (tp,fp,tn,fn,loss,accuracy,precision,recall,auc,prc))

Single server classic ml evaluation
	tp:	943
	fp:	13020
	tn:	31699
	fn:	418

	loss:	0.601757
	acc:	0.708377
	prec:	0.067536
	rec:	0.692873
	auc:	0.762799
	prc:	0.083501


In [40]:
#write to results array
results[0] = tp
results[1][0] = fp
results[2][0] = tn
results[3][0] = fn
results[4][0] = accuracy
results[5][0] = precision
results[6][0] = recall
results[7][0] = auc
results[8][0] = prc
results[9][0] = epochs
results[10][0] = 1
results[11][0] = 1
results[12][0] = 1

In [24]:
#remove data from memory that isn't needed
for_removal = [trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y]
del(for_removal)
del(model)
del(central_server_model)
gc.collect()

2265

## Federated machine learning

The Flower federated learning framework is used from here down. More info can be found at https://flower.dev

In [25]:
#TODO: there is a random out-of-bounds error with masking, for now if it occurs run this cell again
#load the dataset for federated learning
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(federated_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/121.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/120.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/119.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (460

In [26]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    tps = [num_examples * m["tp"] for num_examples, m in metrics]
    fps = [num_examples * m["fp"] for num_examples, m in metrics]
    tns = [num_examples * m["tn"] for num_examples, m in metrics]
    fns = [num_examples * m["fn"] for num_examples, m in metrics]
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    precisions = [num_examples * m["precision"] for num_examples, m in metrics]
    recalls = [num_examples * m["recall"] for num_examples, m in metrics]
    aucs = [num_examples * m["auc"] for num_examples, m in metrics]
    prcs = [num_examples * m["prc"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"tp": sum(tps) / sum(examples),
            "fp": sum(fps) / sum(examples),
            "tn": sum(tns) / sum(examples),
            "fn": sum(fns) / sum(examples),
            "accuracy": sum(accuracies) / sum(examples),
            "precision": sum(precisions) / sum(examples),
            "recall": sum(recalls) / sum(examples),
            "auc": sum(aucs) / sum(examples),
            "prc": sum(prcs) / sum(examples)
           }

In [28]:
NUM_CLIENTS = 24
epochs = fl_epochs

#client_resources = {"num_cpus": 2}
#if tf.config.get_visible_devices("GPU"):
 #   client_resources["num_gpus"] = 1

def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    """Centralized evaluation function"""
    model = make_model()
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
    #model.build(input_shape=(BATCH_SIZE, 28, 28, 1))
    model.set_weights(parameters)
#    loss, accuracy = model.evaluate(central_testloaders_x, central_testloaders_y, batch_size=32, verbose=0)
#    return loss, {"accuracy": accuracy}
    loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc  = model.evaluate(central_testloaders_x[0], central_testloaders_y[0], verbose=2)
    return loss, {"tp": tp,
                  "fp": fp,
                  "tn": tn,
                  "fn": fn,
                  "accuracy": accuracy,
                  "precision": precision,
                  "recall": recall,
                  "auc": auc,
                  "prc": prc
                 }


strategy = fl.server.strategy.FedAvg(
    fraction_fit=fraction_fit,  # Sample 10% of available clients for training
    fraction_evaluate=fraction_evaluate,  # Sample 5% of available clients for evaluation
    min_fit_clients=7,  # Never sample less than 10 clients for training
    min_evaluate_clients=6,  # Never sample less than 5 clients for evaluation
    min_available_clients=7,  # Wait until at least 75 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate
)

# Start simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    #client_resources=client_resources,
)

INFO flwr 2023-06-06 20:53:42,380 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2023-06-06 20:53:45,285	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-06-06 20:53:46,554 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 16244681932.0, 'memory': 32489363867.0, 'CPU': 8.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 16244681932.0, 'memory': 32489363867.0, 'CPU': 8.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-06-06 20:53:46,557 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-06 20:53:46,560 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


(launch_and_get_parameters pid=10317) 
(launch_and_get_parameters pid=10317) This is client:  8
(launch_and_get_parameters pid=10317) Loaded data for client:  8 
(launch_and_get_parameters pid=10317) 
(launch_and_get_parameters pid=10317) Making model:  8


INFO flwr 2023-06-06 20:53:53,251 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one random client
INFO flwr 2023-06-06 20:53:53,257 | server.py:88 | Evaluating initial parameters
INFO:flwr:Evaluating initial parameters


(launch_and_get_parameters pid=10317) 
(launch_and_get_parameters pid=10317) Client CID: 8 is done.
(launch_and_get_parameters pid=10317) 
1440/1440 - 13s - loss: 0.6332 - tp: 15.0000 - fp: 2035.0000 - tn: 42684.0000 - fn: 1346.0000 - accuracy: 0.9266 - precision: 0.0073 - recall: 0.0110 - auc: 0.3371 - prc: 0.0201 - 13s/epoch - 9ms/step


INFO flwr 2023-06-06 20:54:06,751 | server.py:91 | initial parameters (loss, other metrics): 0.6331724524497986, {'tp': 15.0, 'fp': 2035.0, 'tn': 42684.0, 'fn': 1346.0, 'accuracy': 0.9266275763511658, 'precision': 0.007317073177546263, 'recall': 0.011021307669579983, 'auc': 0.33707618713378906, 'prc': 0.020078491419553757}
INFO:flwr:initial parameters (loss, other metrics): 0.6331724524497986, {'tp': 15.0, 'fp': 2035.0, 'tn': 42684.0, 'fn': 1346.0, 'accuracy': 0.9266275763511658, 'precision': 0.007317073177546263, 'recall': 0.011021307669579983, 'auc': 0.33707618713378906, 'prc': 0.020078491419553757}
INFO flwr 2023-06-06 20:54:06,754 | server.py:101 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-06-06 20:54:06,757 | server.py:218 | fit_round 1: strategy sampled 12 clients (out of 24)
DEBUG:flwr:fit_round 1: strategy sampled 12 clients (out of 24)


...

1440/1440 - 14s - loss: 0.5545 - tp: 18216.0000 - fp: 245005.0000 - tn: 694094.0000 - fn: 10365.0000 - accuracy: 0.7361 - precision: 0.0692 - recall: 0.6373 - auc: 0.7492 - prc: 0.0833 - 14s/epoch - 10ms/step


INFO flwr 2023-06-06 22:25:10,526 | server.py:119 | fit progress: (20, 0.5544987916946411, {'tp': 18216.0, 'fp': 245005.0, 'tn': 694094.0, 'fn': 10365.0, 'accuracy': 0.7361007928848267, 'precision': 0.06920420378446579, 'recall': 0.6373465061187744, 'auc': 0.7492278814315796, 'prc': 0.08327484130859375}, 5463.768704083)
INFO:flwr:fit progress: (20, 0.5544987916946411, {'tp': 18216.0, 'fp': 245005.0, 'tn': 694094.0, 'fn': 10365.0, 'accuracy': 0.7361007928848267, 'precision': 0.06920420378446579, 'recall': 0.6373465061187744, 'auc': 0.7492278814315796, 'prc': 0.08327484130859375}, 5463.768704083)
DEBUG flwr 2023-06-06 22:25:10,530 | server.py:168 | evaluate_round 20: strategy sampled 6 clients (out of 24)
DEBUG:flwr:evaluate_round 20: strategy sampled 6 clients (out of 24)


(launch_and_evaluate pid=10317) 
(launch_and_evaluate pid=10317) This is client:  13
(launch_and_evaluate pid=10317) Loaded data for client:  13 
(launch_and_evaluate pid=10317) 
(launch_and_evaluate pid=10317) Making model:  13
(launch_and_fit pid=10317) 43/43 - 2s - loss: 0.4954 - tp: 507.0000 - fp: 142.0000 - tn: 541.0000 - fn: 176.0000 - accuracy: 0.7672 - precision: 0.7812 - recall: 0.7423 - auc: 0.8433 - prc: 0.8403 - val_loss: 0.5401 - val_tp: 270.0000 - val_fp: 94.0000 - val_tn: 263.0000 - val_fn: 87.0000 - val_accuracy: 0.7465 - val_precision: 0.7418 - val_recall: 0.7563 - val_auc: 0.8047 - val_prc: 0.7905 - 2s/epoch - 47ms/step [repeated 2x across cluster]
(launch_and_fit pid=10317) Making model:  13
(launch_and_evaluate pid=10317) Client CID: 13 is done.
(launch_and_evaluate pid=10317) 1440/1440 - 21s - loss: 0.5768 - tp: 232.0000 - fp: 11849.0000 - tn: 33851.0000 - fn: 148.0000 - accuracy: 0.7396 - precision: 0.0192 - recall: 0.6105 - auc: 0.7465 - prc: 0.0243 - 21s/epoch -

DEBUG flwr 2023-06-06 22:26:34,745 | server.py:182 | evaluate_round 20 received 6 results and 0 failures
DEBUG:flwr:evaluate_round 20 received 6 results and 0 failures
INFO flwr 2023-06-06 22:26:34,754 | server.py:147 | FL finished in 5547.997309826
INFO:flwr:FL finished in 5547.997309826
INFO flwr 2023-06-06 22:26:34,762 | app.py:218 | app_fit: losses_distributed [(1, 0.6521012654027837), (2, 0.6377602036621954), (3, 0.6012001252486064), (4, 0.5913825548425851), (5, 0.5886446325091276), (6, 0.6036289297526607), (7, 0.5795185670857499), (8, 0.5711905124132092), (9, 0.5941266029678768), (10, 0.5669200954532216), (11, 0.5780689555261936), (12, 0.5749206990912193), (13, 0.5453178723069628), (14, 0.5616264538186503), (15, 0.5639194761486735), (16, 0.5456234453446621), (17, 0.5520775648926415), (18, 0.5769246194901456), (19, 0.5561362638265712), (20, 0.5669301139731561)]
INFO:flwr:app_fit: losses_distributed [(1, 0.6521012654027837), (2, 0.6377602036621954), (3, 0.6012001252486064), (4, 0.5

(launch_and_evaluate pid=10317) 1440/1440 - 15s - loss: 0.5610 - tp: 321.0000 - fp: 10929.0000 - tn: 34680.0000 - fn: 150.0000 - accuracy: 0.7596 - precision: 0.0285 - recall: 0.6815 - auc: 0.7864 - prc: 0.0339 - 15s/epoch - 10ms/step [repeated 2x across cluster]
(launch_and_evaluate pid=10317)  [repeated 4x across cluster]


In [42]:
#TODO: This whole cell needs to be cleaned up
# Write out the results of evaluation.

count = 0
for key in (history.metrics_distributed):
    #print(key, '->', history.metrics_distributed[key][2:][0][1])
    results[count][1] = history.metrics_distributed[key][2:][0][1]
    count += 1
count = 0
results[9][1] = fl_epochs
results[10][1] = num_rounds
results[11][1] = NUM_CLIENTS * fraction_fit
results[12][1] = NUM_CLIENTS * fraction_evaluate
for key in (history.metrics_centralized):
    results[count][2] = history.metrics_centralized[key][2:][0][1]
    count += 1
results[9][2] = fl_epochs
results[10][2] = num_rounds
results[11][2] = NUM_CLIENTS * fraction_fit
results[12][2] = NUM_CLIENTS * fraction_evaluate
file = results_path + "history" + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"
results_out = pd.DataFrame(results)
results_out.columns = ['single_server', 'distributed', 'centralized']
results_out.index = ['tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc', 'prc', 'epochs', 'rounds', 'train_clients', 'evaluate_clients']
print(results_out)
pd.DataFrame(results_out).to_csv(file)
print("\nWriting out results.")

                  single_server   distributed   centralized
tp                   943.000000    308.865034   2005.000000
fp                 13020.000000  15074.747008  37048.000000
tn                 31699.000000  30565.240840  97109.000000
fn                   418.000000    131.147118   2078.000000
accuracy               0.708377      0.670011      0.716971
precision              0.067536      0.020063      0.051340
recall                 0.692873      0.710020      0.491060
auc                    0.762799      0.734435      0.639574
prc                    0.083501      0.021957      0.051363
epochs                10.000000      3.000000      3.000000
rounds                 1.000000     20.000000     20.000000
train_clients          1.000000     14.400000     14.400000
evaluate_clients       1.000000      7.200000      7.200000

Writing out results.
